# Helpful Links
* https://www.dataquest.io/blog/jupyter-notebook-tips-tricks-shortcuts/

In [4]:
# %load ../src/utils.py
from collections import defaultdict


def printUniqueTokens(series):
    unique_series = series.unique()
    token_count = {}
    for a in unique_series:
        tokens = a.split(' ')
        for t in tokens:
            if t not in token_count:
                token_count[t] = 1
            else:
                token_count[t] += 1

    for key, value in sorted(token_count.items(), key=lambda item: item[1]):
        print("%s: %s" % (key, value))


# Source: https://www.geeksforgeeks.org/python-merge-list-with-common-elements-in-a-list-of-lists/
# merge function to merge all sublist having common elements.
def merge_common(lists):
    neigh = defaultdict(set)
    visited = set()
    for each in lists:
        for item in each:
            neigh[item].update(each)

    def comp(node, neigh=neigh, visited=visited, vis=visited.add):
        nodes = {node}
        next_node = nodes.pop
        while nodes:
            node = next_node()
            vis(node)
            nodes |= neigh[node] - visited
            yield node

    for node in neigh:
        if node not in visited:
            yield sorted(comp(node))


In [13]:
# %load ../src/clean.py
from typing import List, Any, Set, Tuple

import pandas as pd
import textdistance as td
import json
from itertools import combinations


def calcDistances(strings: Set[str], completelyInsideOtherBias: float = 0.7,
                  algo: str = "jaccard", readFromFile: bool = True, writeToFile: bool = True, doBias: bool = True)\
        -> List[Tuple[str, str, float]]:
    """Calculates the distanced according to the algorithm in the constant variable algo.

    If the constant doBias is set to true, then the bias function is applied with the parameter
    completelyInsideOtherBias. If the constant readFromFile is set to true, then the algorithm won't execute, but
    rather load a cached representation from a json file, which will not work for all algorithms, but only those that
    were cached previously

    Args:
        strings: to calculate the distances for each combination
        completelyInsideOtherBias: parameter for the bias function, default = 0.7
        algo: to use for text distance comparison, default = "jaccard"
        readFromFile: if a cached text distance matrix should be read from a file, default = True
        writeToFile: if the calculated text distance matrix should be written to a file, default = True
        doBias: if the bias function should be applied, default = True

    Returns:
        list of tuples with the form (name1: String, name2: String, distanceValue: float)
    """

    def bias(s1, s2):
        if s1 in s2 or s2 in s1:
            return completelyInsideOtherBias
        else:
            return 0

    stringCombinations = set(map(frozenset, combinations(set(strings), 2)))

    if readFromFile:
        with open(PATH_PREFIX + "/" + algo + ".json", "r") as file:
            allDistances = json.loads(file.read())
    else:
        allDistances = [(s1, s2, s1 == s2 and -1 or td.jaccard(s1.split(), s2.split())) for s1, s2 in stringCombinations]
        allDistances.sort(key=lambda x: x[2], reverse=True)
        if writeToFile:
            with open(PATH_PREFIX + "/" + algo + ".json", "w+") as file:
                file.write(json.dumps(allDistances))

    if doBias:
        for i in range(len(allDistances)):
            allDistances[i] = (allDistances[i][0], allDistances[i][1],
                               allDistances[i][2] + bias(allDistances[i][0], allDistances[i][1]))
    return allDistances


def convertToEqualityRings(distanceList: List[Tuple[str, str, float]]) -> List[List[str]]:
    """Converts a distance list of Tuples(str,str,float) into a list of sets with common elements merged.

    Example:
        convertToEqualityRings([("hello", "hellas", 0.8), ("hello", "bye", 0), ("test1", "test2", 0.8)])
        will return: [{"hello", "hellas", "bye"},{"test1", "test2"}]

    Args:
        distanceList: contains tuples of form: (name1: String, name2: String, distanceValue: float).
            The distanceValue is ignored and just the names are taken in to account, which are then used to merge all
            Tuples of form (name1: String, name2: String) into sets if they have any elements in common.

    Returns:
        a list of sets with common elements merged
    """
    sets = map(lambda x: set(x[0:2]), distanceList)
    return list(merge_common(sets))


def loadGoldStandard() -> Tuple[List[Set[int]], List[int]]:
    """Loads the Gold Standard from the file provided by the HPI 'restaurants_DPL.tsv'

    Returns:
        a tuple with two lists (dupesets, dupeids).
            dupesets is a list of all rows in the Gold Standard as sets
            dupeids is a list of all ids that are in Gold Standard
    """
    dupedf = pd.read_csv(PATH_PREFIX + '/restaurants_DPL.tsv', delimiter='\t')
    dupedict = {}
    for i, dupeRow in dupedf.iterrows():
        if dupeRow[0] not in dupedict:
            dupedict[dupeRow[0]] = set()
            dupedict[dupeRow[0]].add(dupeRow[0])
        # dupedict[dupeRow[0]].add(dupeRow[0])
        dupedict[dupeRow[0]].add(dupeRow[1])

    dupesets: List[Set[Any]] = list(dupedict.values())
    dupeids: List[int] = [y for x in dupesets for y in x]
    return dupesets, dupeids


class Statics:
    """
    Class that contains important static values.
    """
    CITY_REPLACE_DICT = {
        r"w. hollywood": "west hollywood",
        r"new york city": "new york",
        r"west la": "los angeles",
        r"la": "los angeles"
    }
    ADDRESS_REPLACE_DICT = {
        r"(ave|av)": "ave",
        r"(blvd|blv)": "blvd",
        r"(sts)": "st",
        r"s\.": "s",
        r" ?between.*$": ""
    }
    BRACKETS_REGEX = r" ?\(.*\)"
    NON_ALPHA_OR_SPACE_REGEX = r"[^a-zA-Z0-9 ]"
    NON_ALPHA_REGEX = r"[^a-zA-Z0-9]"


def preProcess(df: pd.DataFrame) -> pd.DataFrame:
    """Pre processes the given DataFrame by applying a lot of Regex replacements.

    Args:
        df: a pandas DataFrame to pre process

    Returns:
        a pre processed pandas DataFrame
    """
    # reformat the index
    # df = df.drop(labels=['id'], axis=1)
    # df.set_index("id", inplace=True)

    for k, v in Statics.ADDRESS_REPLACE_DICT.items():
        df.address = df.address.str.replace(k, v, case=False, regex=True)
    for k, v in Statics.CITY_REPLACE_DICT.items():
        df.city = df.city.str.replace(k, v, case=False, regex=True)
    df.type = df.type.fillna("")
    df.type = df.type.str.replace(Statics.BRACKETS_REGEX, '', case=False) \
        .str.replace(r"^.*[0-9] ?", "", case=False)
    df.phone = df.phone.str.replace(Statics.NON_ALPHA_REGEX, '', case=False)
    df.name = df.name.str.replace(Statics.BRACKETS_REGEX, '', case=False)

    df["cname"] = df.name.copy()
    df["caddress"] = df.address.copy()
    df.cname = df.cname.str.replace(Statics.NON_ALPHA_OR_SPACE_REGEX, '', case=False) \
        .str.replace('  the$', '', case=False)
    df.caddress = df.caddress.str.replace(Statics.NON_ALPHA_OR_SPACE_REGEX, '', case=False)
    return df


def compareToGold(df: pd.DataFrame, dupesets: List[Set[int]], dupeids: List[int]) -> Tuple[Set, Set, Set, Set, List]:
    """Compares the given DataFrame to the given Gold Standard and prints and returns the results

    Args:
        df: a pandas DataFrame to compare to the Gold Standard
        dupesets: the Gold Standard dupesets, see #loadGoldStandard()
        dupeids: the Gold Standard dupeids, see #loadGoldStandard()

    Returns:
        A Tuple (true_positive, false_negative, false_positive, true_negative, listofparams)
    """
    recognizedDuplicates = list(df[df.id.map(len) > 1].id)
    recognizedNonDuplicates = list(df[df.id.map(len) <= 1].id)
    true_positive = set()
    true_negative = set()
    false_negative = set()
    false_positive = set()
    for dupeset in dupesets:
        if dupeset in recognizedDuplicates:
            true_positive.add(frozenset(dupeset))
        else:
            false_negative.add(frozenset(dupeset))
    for recdup in recognizedDuplicates:
        if recdup in dupesets:
            true_positive.add(frozenset(recdup))
        else:
            false_positive.add(frozenset(recdup))
    for recnondup in recognizedNonDuplicates:
        if recnondup not in dupeids:
            true_negative.add(frozenset(recnondup))

    # times 2 because we work with sets of 2
    tp = len([e for s in true_positive for e in s])
    tn = len([e for s in true_negative for e in s])
    fn = len([e for s in false_negative for e in s])
    fp = len([e for s in false_positive for e in s])
    print("True positives: " + str(tp))
    print("True negatives: " + str(tn))
    print("False positives: " + str(fp))
    print("False negatives: " + str(fn))
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    fscore = (2 * precision * recall) / (precision + recall)
    print("Precision: " + str(precision))
    print("Recall: " + str(recall))
    print("Fscore: " + str(fscore))
    listofparams = [tp, tn, fp, fn, precision, recall, fscore]
    return true_positive, false_negative, false_positive, true_negative, listofparams


def __findClosestEqRingMatch(eqRing: List[List[str]], searchString: str) -> str:
    for i, ring in enumerate(eqRing):
        for e in ring:
            if e == searchString:
                return ring[0]
    return searchString


def dedupe(df: pd.DataFrame, eqRing: List[List[str]]) -> pd.DataFrame:
    """Deduplicates the given preprocessed DataFrame by using the eqRing.

    Args:
        df: a preprocessed pandas DataFrame
        eqRing: to use for grouping by

    Returns:
        a deduped pandas DataFrame
    """
    df["tdkey"] = df.cname.copy()
    df["tdkey"] = df["tdkey"].apply(lambda s: __findClosestEqRingMatch(eqRing, s))
    df = df.groupby(["phone", "tdkey"]).agg(set).reset_index()
    return df


eqRing = []


def clean(df: pd.DataFrame, completelyInsideOtherBias: float = 0.7, filterCutoff: float = 0.65,
          algo: str = "jaccard", readFromFile: bool = True, writeToFile: bool = True, doBias: bool = True) -> pd.DataFrame:
    """Main function to completely clean a restaurant dataset.

    Args:
        df: a pandas DataFrame
        completelyInsideOtherBias: float parameter for the bias function
        filterCutoff: float parameter specifying at which distance value to cut off the distance list

    Returns:
        a deduplicated pandas DataFrame
    """
    global eqRing
    df = preProcess(df)
    distances = calcDistances(df.cname.unique(), completelyInsideOtherBias, algo, readFromFile, writeToFile, doBias)
    filteredDistances = list(filter(lambda x: x[2] >= filterCutoff, distances))
    eqRing = convertToEqualityRings(filteredDistances)
    return dedupe(df, eqRing)


def __firstOfSet(s: Any) -> Any:
    if isinstance(s, set):
        return s.pop()
    else:
        return s


def prepareUploadJsons(df: pd.DataFrame) -> pd.DataFrame:
    """Prepares and saves two json files that can be imported into mongodb.
        Look in ../data/work/deduped_raw.json and ../data/work/deduped_clean.json

    Args:
        df: a pandas DataFrame

    Returns:
        a pandas DataFrame
    """
    df.to_json(PATH_PREFIX + '/deduped_raw.json', orient='records')
    df.name = df.name.apply(__firstOfSet)
    df.address = df.address.apply(__firstOfSet)
    df.city = df.city.apply(__firstOfSet)
    df.cname = df.cname.apply(__firstOfSet)
    df.caddress = df.caddress.apply(__firstOfSet)
    df.to_json(PATH_PREFIX + '/deduped_clean.json', orient='records')
    print("Written two jsons 'deduped_raw.json' and 'deduped_clean.json' to directory '"
          + PATH_PREFIX + "'.\nYou can use those with mongoimport!")
    return df


PATH_PREFIX = "../data"

if __name__ == '__main__':
    dupesets, dupeids = loadGoldStandard()
    dataframe = pd.read_csv(PATH_PREFIX + '/restaurants.tsv', delimiter='\t')
    cleaned = clean(dataframe, 0.5, 0.45, readFromFile=False, writeToFile=False)
    a, b, c, d, e = compareToGold(cleaned, dupesets, dupeids)
    print(b)
    # prepared = prepareUploadJsons(cleaned)

# CODE USED FOR GENERATING THE HEATMAP CHART
#    bias = 0.7
#    cutoff = 0.7
#    allparams = []
#    for bias in np.arange(0.0,1.1,0.1):
#        for cutoff in np.arange(0.0,1.1,0.1):
#            cleaned = clean(dataframe, bias, cutoff)
#            a,b,c,d,params = compareToGold(cleaned, dupesets, dupeids)
#            print(bias,cutoff,*params, sep=",")
#            allparams.append([bias,cutoff,*params])
#    print("##### ALL PARAMS:")
#    for p in allparams:
#        print(*p,sep=",")


True positives: 216
True negatives: 648
False positives: 0
False negatives: 8
Precision: 1.0
Recall: 0.9642857142857143
Fscore: 0.9818181818181818
{frozenset({129, 130}), frozenset({139, 140}), frozenset({141, 142}), frozenset({144, 143})}


# Results
ALGO,tp,tn,fp,fn,precision,recall,fscore

## with bias of 0.7 and cutoff 0.7
jaccard,210,654,0,14,1.0,0.9375,0.967741935483871
overlap,208,637,19,16,0.9162995594713657,0.9285714285714286,0.9223946784922396
tanimoto,176,688,0,48,1.0,0.7857142857142857,0.88
tversky,210,654,0,14,1.0,0.9375,0.967741935483871
monge_elkan,204,660,0,20,1.0,0.9107142857142857,0.9532710280373832

## without bias with cutoff 0.7
jaccard,182,682,0,42,1.0,0.8125,0.896551724137931
overlap,208,637,19,16,0.9162995594713657,0.9285714285714286,0.9223946784922396
tanimoto,176,688,0,48,1.0,0.7857142857142857,0.88
tversky,182,682,0,42,1.0,0.8125,0.896551724137931
monge_elkan,176,688,0,48,1.0,0.7857142857142857,0.88

## with bias of 0.7 and cutoff 0.7

In [15]:
cleaned

,phone,tdkey,id,name,address,city,type,cname,caddress
0,1008138212,indian delights,{809},{indian delights},{3675 satellite blvd.},{duluth},{indian},{indian delights},{3675 satellite blvd}
1,2122060059,big cup,{301},{big cup},{228 8th ave. },{new york},{coffee bar},{big cup},{228 8th ave }
2,2122132288,marnies noodle shop,{751},{sam's noodle shop},{411 third ave.},{new york},{chinese},{sams noodle shop},{411 third ave}
3,2122190500,nobu,{743},{nobu},{105 hudson st.},{new york},{japanese},{nobu},{105 hudson st}
4,2122192777,bistro,"{99, 100}",{montrachet},"{239 w. broadway , 239 w. broadway}",{new york},"{french bistro, french}",{montrachet},"{239 w broadway, 239 w broadway }"
...,...,...,...,...,...,...,...,...,...
751,8188865679,brents deli,{653},{brent's deli},{19565 parthenia ave.},{northridge},{delis},{brents deli},{19565 parthenia ave}
752,8189056515,rubins red hots,{702},{rubin's red hots},{15322 ventura blvd.},{encino},{hot dogs},{rubins red hots},{15322 ventura blvd}
753,8189068881,mulberry st,{692},{mulberry st.},{17040 ventura blvd.},{encino},{pizza},{mulberry st},{17040 ventura blvd}
754,8189854669,cadel sol,{232},{ca'del sol},{4100 cahuenga blvd.},{los angeles},{italian},{cadel sol},{4100 cahuenga blvd}


In [13]:
dupesets, dupeids = loadGoldStandard()

In [16]:
df = pd.read_csv(PATH_PREFIX + '/restaurants.tsv', delimiter='\t')
df = preProcess(df)
#df = df.drop(["name", "address"], axis=1)
df.describe(include=['object'])

,name,address,city,phone,type,cname,caddress
count,864,864,864,864,864,864,864
unique,764,764,46,748,75,759,764
top,spago,3570 las vegas blvd. s,new york,4042372700,american,palm,3570 las vegas blvd s
freq,3,6,338,4,180,3,6


In [43]:
byphone = df.groupby("phone").agg(set).reset_index()
for row in combinations(df.itertuples(),2):
    print(row)
    break

arnie morton's of chicago


In [11]:
df = pd.read_csv('/data/restaurants.tsv', delimiter='\t')
cleaned = clean(df, 0.7, 0.65)
a, b, c, d, e = compareToGold(cleaned, dupesets, dupeids)
missingIds = [y for x in b for y in x]
falseIds = [y for x in c for y in x]

In [61]:
cc = cleaned.copy()
cc.city = cc.city.map(lambda x: next(iter(x)))
cc = cc.groupby(["city", "tdkey"]).agg(list).reset_index()
cc

,city,tdkey,phone,id,name,address,type,cname,caddress
0,atlos angelesnta,103 west,[4042335993],[{787}],[{103 west}],[{103 w. paces ferry rd.}],[{continental}],[{103 west}],[{103 w paces ferry rd}]
1,atlos angelesnta,abbey,[4048768532],[{492}],[{abbey}],[{163 ponce de leon ave.}],[{international}],[{abbey}],[{163 ponce de leon ave}]
2,atlos angelesnta,abruzzi,[4042618186],"[{149, 150}]",[{abruzzi}],"[{2355 peachtree rd. ne, 2355 peachtree rd. p...",[{italian}],[{abruzzi}],"[{2355 peachtree rd ne, 2355 peachtree rd pea..."
3,atlos angelesnta,adrianos ristorante,"[4042372663, 4042372700, 4042377601, 404261701...","[{153, 154}, {181, 182}, {831}, {793}, {496}, ...","[{bone's restaurant, bone's}, {dining room ri...","[{3130 piedmont road, 3130 piedmont rd. ne}, {...","[{american, steakhouses}, {american, internati...","[{bones, bones restaurant}, {dining room ritz...","[{3130 piedmont rd ne, 3130 piedmont road}, {3..."
4,atlos angelesnta,alecks barbecue heaven,[4045252062],[{493}],[{aleck's barbecue heaven}],[{783 martin luther king jr. dr.}],[{barbecue}],[{alecks barbecue heaven}],[{783 martin luther king jr dr}]
...,...,...,...,...,...,...,...,...,...
588,west hollywood,dukes,[3106523100],[{663}],[{duke's}],[{8909 sunset blvd.}],[{coffee shops}],[{dukes}],[{8909 sunset blvd}]
589,westlos angeleske villos angelesge,aja,[8054984049],[{648}],[{baja fresh}],[{3345 kimber dr.}],[{mexican}],[{baja fresh}],[{3345 kimber dr}]
590,westlos angeleske villos angelesge,local nochol,[8187067706],[{684}],[{local nochol}],[{30869 thousand oaks blvd.}],[{health food}],[{local nochol}],[{30869 thousand oaks blvd}]
591,westwood,antonios,[3102091422],[{662}],[{don antonio's}],[{1136 westwood blvd.}],[{italian}],[{don antonios}],[{1136 westwood blvd}]


In [19]:
print(missingIds)
print(falseIds)
print(b)

[129, 130, 179, 180, 139, 140, 144, 143, 121, 122, 141, 142]
[]
{frozenset({129, 130}), frozenset({179, 180}), frozenset({139, 140}), frozenset({144, 143}), frozenset({121, 122}), frozenset({141, 142})}


In [54]:
#df[df.id.isin(missingIds)]
mi = [y for x in b for y in x]
df[df.id.isin([746,747])]

,id,name,address,city,phone,type,cname,caddress
745,746,palm,837 second ave.,new york,2126872953,steakhouses,palm,837 second ave
746,747,palm too,840 second ave.,new york,2126975198,steakhouses,palm too,840 second ave


In [57]:
cleaned

,phone,tdkey,id,name,address,city,type,cname,caddress
0,1008138212,indian delights,{809},{indian delights},{3675 satellite blvd.},{},{indian},{indian delights},{3675 satellite blvd}
1,2122060059,big cup,{301},{big cup},{228 8th ave. },{},{coffee bar},{big cup},{228 8th ave }
2,2122132288,marnies noodle shop,{751},{sam's noodle shop},{411 third ave.},{},{chinese},{sams noodle shop},{411 third ave}
3,2122190500,bouterin,{743},{nobu},{105 hudson st.},{},{japanese},{nobu},{105 hudson st}
4,2122192777,alons at the terrace,"{99, 100}",{montrachet},"{239 w. broadway, 239 w. broadway }",{},"{french bistro, french}",{montrachet},"{239 w broadway, 239 w broadway }"
...,...,...,...,...,...,...,...,...,...
753,8188865679,ambassador grill,{653},{brent's deli},{19565 parthenia ave.},{},{delis},{brents deli},{19565 parthenia ave}
754,8189056515,alons at the terrace,{702},{rubin's red hots},{15322 ventura blvd.},{},{hot dogs},{rubins red hots},{15322 ventura blvd}
755,8189068881,mulberry st,{692},{mulberry st.},{17040 ventura blvd.},{},{pizza},{mulberry st},{17040 ventura blvd}
756,8189854669,cadel sol,{232},{ca'del sol},{4100 cahuenga blvd.},{},{italian},{cadel sol},{4100 cahuenga blvd}
